# TonlibClient example

In [1]:
import sys
sys.path.insert(0, '/app')

import logging
import requests
import asyncio

from pytonlib import TonlibClient


logging.basicConfig(format='%(asctime)s %(module)-15s %(message)s',
                    level=logging.INFO)

Reading latest config:

In [2]:
ton_config_url = 'https://newton-blockchain.github.io/global.config.json'
ton_config = requests.get(ton_config_url).json()

Creating TonlibClient with single LiteServer. You can select LiteServer by index in config file:

In [3]:
loop = asyncio.get_running_loop()
client = TonlibClient(ls_index=0, # choose LiteServer to connect
                      config=ton_config,
                      keystore='/tmp/ton_keystore',
                      loop=loop)

await client.init()

[ 4][t 0][2022-05-06 20:36:00.314293800][Client.cpp:78][&tonlib_requests]	Begin to wait for updates with timeout 1.000000
[ 4][t 1][2022-05-06 20:36:00.320407500][TonlibClient.cpp:1477][!Tonlib][&tonlib_query]	Tonlib got query [id:1] setLogVerbosityLevel {
  new_verbosity_level = 0
}
[ 4][t 1][2022-05-06 20:36:00.321866900][TonlibClient.cpp:1516][!Tonlib][&tonlib_query]	Tonlib got static query setLogVerbosityLevel {
  new_verbosity_level = 0
}
2022-05-06 20:36:00,331 client          TonLib #000 inited successfully


Reading masterchain info:

In [4]:
masterchain_info = await client.get_masterchain_info()
masterchain_info

{'@type': 'blocks.masterchainInfo',
 'last': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '-9223372036854775808',
  'seqno': 20361208,
  'root_hash': 'FDEyd4nZgolZ9ryWCK32u/5MsPCDTo6qJHT3XGFF8XA=',
  'file_hash': 'Jn9q+fqs6dF5wc+3DzjCsQdGL4g8IfIr9RpYnS1QwsA='},
 'state_root_hash': 'xQvdczD+gdaTamtLDIo9rwxLG3w4fchSiEcnaPsHuN0=',
 'init': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '0',
  'seqno': 0,
  'root_hash': 'F6OpKZKqvqeFp6CQmFomXNMfMj2EnaUSOXN+Mh+wVWk=',
  'file_hash': 'XplPz01CXAps5qeSWUtxcyBfdAo5zVb1N979KLSKD24='},
 '@extra': '1651869370.3371992:0:0.507966611324214'}

Block header for the last masterchain block:

In [5]:
block_header = await client.get_block_header(**masterchain_info['last'])
block_header

{'@type': 'blocks.header',
 'id': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '-9223372036854775808',
  'seqno': 20361208,
  'root_hash': 'FDEyd4nZgolZ9ryWCK32u/5MsPCDTo6qJHT3XGFF8XA=',
  'file_hash': 'Jn9q+fqs6dF5wc+3DzjCsQdGL4g8IfIr9RpYnS1QwsA='},
 'global_id': -239,
 'version': 0,
 'flags': 1,
 'after_merge': False,
 'after_split': False,
 'before_split': False,
 'want_merge': True,
 'want_split': False,
 'validator_list_hash_short': 122883420,
 'catchain_seqno': 306940,
 'min_ref_mc_seqno': 20361205,
 'is_key_block': False,
 'prev_key_block_seqno': 20351399,
 'start_lt': '27699743000000',
 'end_lt': '27699743000004',
 'gen_utime': 1651869354,
 'vert_seqno': 1,
 'prev_blocks': [{'@type': 'ton.blockIdExt',
   'workchain': -1,
   'shard': '-9223372036854775808',
   'seqno': 20361207,
   'root_hash': 'AM+LcJOyfGSm1dpbaQDXpOgY7bcMcEDawtCAUwcJLJw=',
   'file_hash': 'AyW+wJSoJZfbBB3Y1JrgJ19SKlpC7WvitkeVk/yplYI='}],
 '@extra': '1651869370.6080866:0:0.17246900748102634'}

Shards of the masterchain block:

In [6]:
shards = await client.get_shards(master_seqno=masterchain_info['last']['seqno'])
shards

{'@type': 'blocks.shards',
 'shards': [{'@type': 'ton.blockIdExt',
   'workchain': 0,
   'shard': '-9223372036854775808',
   'seqno': 25532997,
   'root_hash': 'JI8s3H5c7g4Vexlezl6V+xhvKYjIZKsA8ItgZHJdtQU=',
   'file_hash': 'cMglRNmDveIczi8SjzIsBGHDT0baUa+bwe1ba5Qh7CI='}],
 '@extra': '1651869370.8817048:0:0.3841969484094021'}

Read transactions from the shard block:

In [7]:
txs = await client.get_block_transactions(**masterchain_info['last'], count=10)

print('Is incomplete:', txs['incomplete'])
print('Num txs:', len(txs['transactions']))

Is incomplete: False
Num txs: 5


Read transaction details:

In [8]:
tx = txs['transactions'][0]
tx

{'@type': 'blocks.shortTxId',
 'mode': 135,
 'account': '-1:3333333333333333333333333333333333333333333333333333333333333333',
 'lt': '27699743000001',
 'hash': 'jOIVA+yIkoPywpjrFj9VYTohez2au0ooow5uPLYWFAc='}

In [9]:
await client.get_transactions(**tx, limit=1)

[{'@type': 'raw.transaction',
  'address': {'@type': 'accountAddress',
   'account_address': 'Ef8zMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzM0vF'},
  'utime': 1651869357,
  'data': 'te6cckECBwEAAYkAA69zMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzAAAZMVlmyAIoy6aZP4NZkz72hGeQDNHm2NpTPKWaFnx4LB9YizUu2gAAGTFZZsgBYnWGrQABQIAQIDAQGgBACCco+Ec6Mj7xtio/Duyx50cd7VDh/dyMx2l/MOpAqkYJIU5/6pVPLSgialzCVl4c6EkwC2lWh3d1HX8y6moQPwkl4CDwQJKU/jnFgRBQYAq2n+AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE/zMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzSlP45xAAAADJiss2QAMTrDVpAAJ5CYUwQ6+AAAAAAAAAAAGQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFvAAAAAAAAAAAAAAAABLUUtpEnlC4z33SeGHxRhIq/htUa7i3D8ghbwxhQTn44EwPyXiw==',
  'transaction_id': {'@type': 'internal.transactionId',
   'lt': '27699744000002',
   'hash': 'AnL1aJnySYYOG+Godrlio8NhYRZO3avoLzfkdXpnnOo='},
  'fee': '0',
  'storage_fee': '0',
  'other_fee': '0',
  'in_msg': {'@type': 'raw.message',
   'source': 'Ef8AAAAAAAAAAAAAAAA